In [ ]:
import pandas as pd
import re
import torch
import os
from transformers import GPT2Tokenizer, GPT2LMHeadModel, set_seed, get_linear_schedule_with_warmup
from tqdm import tqdm, trange
import transformers
import accelerate
from torch.utils.data import Dataset, DataLoader

In [ ]:
data = pd.read_csv("data/goodreads_books.csv", converters={'COLUMN_NAME': pd.eval})

In [ ]:
data = data.filter(["title", "rating_count", "average_rating", "genre_and_votes"], axis=1)

In [ ]:
def genre_break(genre):
    for index, item in enumerate(genre):
        new_item = re.sub('\s[0-9]+', '', item)
        genre[index] = new_item
    
    return genre

In [ ]:
data["genre"] = data["genre_and_votes"].str.split(", ")
data.drop(axis=1, columns=["genre_and_votes"], inplace=True)

In [ ]:
data.dropna(inplace=True, axis=0)

In [ ]:
data.info()

In [ ]:
data["genre"].apply(lambda item: genre_break(item))

In [ ]:
data

## Fine-Tuning Set-Up

In [ ]:
class BookTitles(Dataset):
    def __init__(self, content, gpt2_type="gpt2", max_length=256):

        self.tokenizer = GPT2Tokenizer.from_pretrained(gpt2_type, bos_token = '<startoftext>', eos_token = '<endoftext>', pad_token = '<pad>')
        self.input_ids = []

        for _, row in content.iterrows():
          prep = self.tokenizer.encode(f'<startoftext>Tags: {row[3]}\nTitle: {row[0]}<endoftext>', truncation = True, max_length = max_length, padding="max_length")
          self.input_ids.append(torch.tensor(prep))
        
    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, item):
        return self.input_ids[item]

In [ ]:
def pack_tensor(new_tensor, packed_tensor, max_seq_len):
    if packed_tensor is None:
        return new_tensor, True, None
    if new_tensor.size()[1] + packed_tensor.size()[1] > max_seq_len:
        return packed_tensor, False, new_tensor
    else:
        packed_tensor = torch.cat([new_tensor, packed_tensor[:, 1:]], dim=1)
        return packed_tensor, True, None

In [ ]:
def train(
    dataset, model, tokenizer,
    batch_size=16, epochs=4, lr=2e-5,
    max_seq_len=400, warmup_steps=200,
    gpt2_type="gpt2", output_dir=".", output_prefix="wreckgar",
    test_mode=False, save_model_on_epoch=False,
):
    acc_steps = 100
    device = torch.device("cuda")
    model = model.cuda()
    model.train()

    optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
    scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=warmup_steps, num_training_steps=-1
    )

    train_dataloader = DataLoader(dataset, batch_size=1, shuffle=True)
    loss=0
    accumulating_batch_count = 0
    input_tensor = None

    for epoch in range(epochs):

        print(f"Training epoch {epoch}")
        print(loss)
        for idx, entry in tqdm(enumerate(train_dataloader)):
            (input_tensor, carry_on, remainder) = pack_tensor(entry, input_tensor, 768)

            if carry_on and idx != len(train_dataloader) - 1:
                continue

            input_tensor = input_tensor.to(device)
            outputs = model(input_tensor, labels=input_tensor)
            loss = outputs[0]
            loss.backward()

            if (accumulating_batch_count % batch_size) == 0:
                optimizer.step()
                scheduler.step()
                optimizer.zero_grad()
                model.zero_grad()

            accumulating_batch_count += 1
            input_tensor = None
        if save_model_on_epoch:
            torch.save(
                model.state_dict(),
                os.path.join(output_dir, f"{output_prefix}-{epoch}.pt"),
            )
    return model

In [ ]:
train_set = BookTitles(data)

## Model Set-Up

In [ ]:
torch.manual_seed(32)
tokenizer = GPT2Tokenizer.from_pretrained('gpt2', bos_token = '<startoftext>', eos_token = '<endoftext>', pad_token = '<pad>')
model = GPT2LMHeadModel.from_pretrained('gpt2').cuda()
model.resize_token_embeddings(len(tokenizer))

In [ ]:
model = train(train_set, model, tokenizer)

In [ ]:
text = "<startoftext>Tags: ['Fantasy']\nTitle: "
encoded_input = tokenizer.encode(f'{text}', return_tensors='pt').to("cuda")
outputs = model.generate(encoded_input, max_length = 40, no_repeat_ngram_size = 2, do_sample = False, num_return_sequences = 0, num_beams = 5, top_k = 50, top_p = 0.90, temperature = 0)

In [ ]:
print(tokenizer.decode(outputs[0], skip_special_tokens = True))